# LSTM - bert128

- **Training set**: LO_SARD102(0.6 split)
- **Test/Validation set:** LO_SARD102 (0.2 split)
- **Features**: bert sequence (128)
- **Class imbalance processing**: Class weights

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from helper_functions import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X = numpy.load('/mnt/md0/user/scheuererra68323/LOSARD102_bert128_X.npy')
y = pd.read_hdf('/mnt/md0/user/scheuererra68323/LOSARD102_bert128_y.h5').is_vulnerable.values

X_train, X_test, Y_train, Y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=0)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, stratify=Y_train, test_size=0.25, random_state=0)

print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)
print(X_test.shape)
print(Y_test.shape)

(30712, 128, 768)
(30712,)
(10238, 128, 768)
(10238,)
(10238, 128, 768)
(10238,)


In [10]:
def run_classifier():
    model = keras.Sequential()
    model.add(layers.LSTM(512, input_shape=(X_train.shape[1], X_train.shape[2]), 
                          return_sequences=True))
    model.add(layers.Dropout(0.1))
    model.add( layers.TimeDistributed(layers.Dense(256)) )
    model.add(layers.Flatten())
    model.add( layers.Dense(128))
    model.add( layers.Dense(64))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.summary()
    model.compile(loss='binary_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy']
                 )
    
    classWeights = {
        0: ( 1 / Counter(Y_train)[0] ) * Y_train.shape[0] / 2.0 ,
        1: ( 1 / Counter(Y_train)[1] ) * Y_train.shape[0] / 2.0
    }
    print(f"\nclassWeights: {classWeights}\n")

    history = model.fit(X_train, Y_train,
                        batch_size=256,
                        epochs=10,
                        verbose=True,
                        validation_data=(X_val, Y_val),
                        class_weight=classWeights
                        )
    
    return model, history


model, history = run_classifier()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128, 512)          2623488   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128, 512)          0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 128, 256)          131328    
_________________________________________________________________
flatten_2 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               4194432   
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                

In [ ]:
def evaluate_classifier(_model, _history):
    Y_pred = numpy.round(_model.predict(X_test))

    print(classification_report(Y_test, Y_pred))
    print_metrics(confusion_matrix(Y_test, Y_pred), Y_test)
    print()
    score = _model.evaluate(X_test, Y_test, verbose=False)
    print('Test loss:\t{:.6f}'.format(score[0]))
    print('Test accuracy:\t{:.6f}'.format(score[1]))

    plot_loss(_history)
    plot_accuracy(_history)


evaluate_classifier(model, history)

In [7]:
model.save('LSTM_bert128_weighted')
model.save('LSTM_bert128_weighted.h5')

INFO:tensorflow:Assets written to: LSTM_bert128_weighted/assets
